# Decision Trees
page 167<br>
See
- https://github.com/ageron/handson-ml/blob/master/06_decision_trees.ipynb for code and further information,
- https://guide.macports.org/#using.common-tasks for the installation of graphviz (here particularly "sudo port install ..." and "sudo port search ..."),
- https://www.graphviz.org and page 168 of the book for instructions on how to use graphviz, and
- https://stackoverflow.com/questions/10628262/inserting-image-into-ipython-notebook-markdown for information on how to include an image in Jupyter Markdown. The relevant command, starting from the folder that contains the Jupyter notebook is *<"XXX">*, where "XXX" is *img src="imagepath"* (see example below).

Like SVMs, decision trees are very versatile and can be used for classifcation as well as for Regression. They are fundamental component of random forests, see chapter 7.
## Training and Visualizing a Decision Tree
page 167<br>
Let's build and display a decision tree so we can better understand how they work!

In [1]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
iris = load_iris()
X = iris.data[:, 2:]
y = iris.target
tree_clf = DecisionTreeClassifier(max_depth=2)
tree_clf.fit(X, y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

The decision tree is trained. Now, let's visualize it using graphviz! The code below first defines the path for saving the file "iris_tree.dot", which will be produced in the following code.

In [2]:
# determine where to save the figures
import os
PROJECT_ROOT_DIR = "."
def image_path(fig_id):
    return os.path.join(PROJECT_ROOT_DIR, "images", fig_id)
# from sklearn.tree, use the export_graphviz command to export the structure of the decision tree into a dot file ...
# ... under the path determined above
from sklearn.tree import export_graphviz
export_graphviz(
        tree_clf,
        out_file=image_path("iris_tree.dot"),
        feature_names=iris.feature_names[2:],
        class_names=iris.target_names,
        rounded=True,
        filled=True
    )

Open a terminal and use the "cd ..." and "ls" commands to navigate to the folder containing the file "iris_tree.dot". Then, run the command "dot -Tpng iris_tree.dot -o iris_tree.png" to have graphviz make a png file out of the dot file. That dot file can be displayed using the code explained at the top of this notebook. 
<img src="images/iris_tree.png">
## Making Predictions
The classification process starts at the top *root node* (depth=0) and proceeds to its child nodes. One of them is a *leaf node*, i.e., it does not split further. The other nodes splits into two child nodes, both of which are leaf nodes. The nodes' attributes' meanings are as follows. The number of samples that fall into the node are specified by *samples*. Their distribution over all possible classes is described by *values*. The *gini* attribute measures the *gini impurity*,
$$G_i=1-\sum_{k=1}^np_{i,k}^2\,,$$
of the node. Here, $p_{i,k}=\text{value}_{k,i}/\text{samples}_i$ is the ratio of class $k$ instances among the training instances of the $i$-th node.<br><br>
**General note**<br>
One of the many qualities of Decision Trees is that they require very little data preparation. In particular, they don't require feature scaling or centering at all.<br><br>
**General note**<br>
Scikit-Learn uses the CART algorithm, which produces only *binary trees*: nonleaf nodes always have two children (i.e., questions only have yes/no answers). However, other algorithms such as ID3 can procuce Decision Trees with nodes that haver more than two children.<br><br>
**Model Interpretation: White Box Versus Black Box**<br>
As you can see Decisison Trees are fairly intuitive and their decisions are easy to interpret. Such models are often called *white box modles*. In contrast, as we will see, Random Forests or neural networks are generally considered *black box models*. They make great predictions, and you can easily check the calculation that they performed to make  these predictions; nevertheless, it is usually hard to explain in simple terms why the predictions were made. For example, if a neural network says that a particular person appears on a picture, it is hard to know what actually contributed to this  prediction: did the model recognize that person's eyes? Her mouth? Her nose? Her shoes? Or even the couch that she was sitting on? Converesy, Decision Trees provide nice and simple classification rules that can even be applied manually if need be (e.g., for flower classification).
## Estimating Class Probabilities
The probability $p_{i,k}=\text{value}_{k,i}/\text{samples}_i$ can not only be used for the Gini impurity but also to output the probability that some - possibly new - instance belongs to a certain class: just go through the trained Decision Tree using the features of the new instance and arrive at node $i$. Then $p_{i,k}$ are the class probabilities for that instance. This is coded as easy as follows.

In [3]:
new_instance = [5, 1.5]                       # new instance with petal length = 5 cm and petal width = 1.5 cm
print(tree_clf.predict_proba([new_instance])) # probabilities for Iris-Setosa, Iris-Versicolor, Iris-Virginica
print(tree_clf.predict([new_instance]))       # classify according to highest probability

[[0.         0.90740741 0.09259259]]
[1]


## The CART Training Algorithm, Computational Complexity, Gini Impurity vs. Entropy, and Regularization Hyperparameters
page 171-174<br>
In each node, the Classification And Regression Tree (CART, standard algorithm for Scikit-Learn) splits the set into two subsets, i.e., there will be two direct child nodes. It always uses only one feature $k$ and splits it for the feature value $t_k$. Both $k$ and $t_k$ are chosen such that the training set is split into the purest possible subsets, as measured by the Gini impurities. The subsets are weighted with their size. So at each step, it tries to minimize the cost function
$$J(k,t_k)=\frac{m_{\rm left}}{m}G_{\rm left}+\frac{m_{\rm right}}{m}G_{\rm right}\,,$$
where $G_{\rm left/right}$ ($m_{\rm left/right}$) is the Gini impurity (number of instances) in the left/right subset. The algorithm stops once it reaches the maximum depth or once it does not manage to reduce the impurity further.<br>
Hyperparameters are *max_depth*, *min_samples_split*, *min_samples_leaf*, *min_weight_fraction_leaf*, and *max_leaf_nodes*. Unfortunately, finding the optimal tree is an $NP$-complete problem and takes $\mathcal{O}(exp(m))$ time ($m$ is the number of instance).<br><br>
**Warning / caution**<br>
As you can see, the CART algrotihm is a *greedy algorithm*: it greedily searces for an optimum split at the top level, then repeats the process at each level. It does not check whether or not the split will lead to the lowest possible impurity several levels down. A greedy algorithm often produces a reasonably good solution, but it is not guaranteed to be the optimal solution.<br><br>
Decision Trees are usually rather balanced so traversing th eDecision Tree requires going through roughly $\mathcal{O}(log_2m)$ nodes. Since each node requires checking only one single feature, ther overall complexity is also $\mathcal{O}(log_2m)$. However, training requires checking all features of all instances at each node and thus has the complexity $\mathcal{O}(n\times m\times log_2m)$. For small training sets up to about a thousand features Scikit-Learn can speed training up by presorting the data (set *presort=True*).<br><br>
By default, the CART algorithm tries to minimize the *Gini impurity* but by setting the hyperparameter *criterion* to *entropy*, one can switch from the Gini impurity to the entropy $H_i$ of the $i$-th node,
$$H_i=-\sum_{p_{i,k}\neq0,k=1}^np_{i,k}\,log\,p_{i,k}\,.$$
$H_i$ will be 0 for $p_{i,k}\in\{0,1\}$ (perfect classification) but greater than zero ($H_i=-log\frac{1}{2}$) for $p_{i,0}=p_{i,1}=1/2$ (example for non-perfect classification). In Shannon's *information theory*, the entropy measures the average information content of a message: entropy is zero when all messages are identical. A reduction of entropy is often called an *information gain*.<br>
Both *Gini impurity* and *entropy* grow similar trees. *Gini impurity* is slightly faster, so it is a good starting point. However, if they differ, *entropy* tends to produce slightly more balanced results.<br><br>
Decision Trees make very few assumptions about the training data. Such models are often called *nonparatmetric models*, because the number of parameters is not determined prior to training. In contrast, a *parametric* model, e.g., a linear model, has a fixed number of parameters prior to training. To counteract overfitting one should use regularization. In Scikit-Learn, regularization can be achieved by restricting/setting the hyperparameters *max_depth* (maximum depth, see figure above), *min_samples_split* (minimum number of instances before splitting), *min_samples_leaf* (minimum number of instances in a leaf node), *min_weight_fraction_leaf* (same as *min_samples_leaf* but for class fractions), *max_leaf_nodes* (obvious), and *max_features* (max number of features randomly considered within a node). Increasing *min_\** or reducing *max_\** hyperparameters will regularize the model, see also Figure 6-3 in the book.<br><br>
**General note**<br>
Other algorithms work by first training the Decision Tree without restrictions, then *pruning* (deleting) unnecessary nodes. A node whose children are all leaf nodes is considered unnecessary if the purity improvement it provides is not *statistically significant*. Standard statistical tests, such as the $\chi^2$ *test*, are used to estimate the probability that the improvement is purely the result of chance (which is called the *null hypothesis*). If this probability, called the *p-value*, is higher than a given threshold (typically 5%, controlled by a hyperparameter), then the node is considered unnecessary and its children are deleted. The pruning continues until all unnecessary nodes have been pruned.
## Regression
page 175<br>
Decision Trees can also be used for regression tasks. Here, we use noisy quadratic data. Instead of minimizing the *Gini impurity* or the *entropy*, the CART algorithm now tries to minimize the MSE on the $y$-coordinates by splitting the data at some $x$-feature and fitting constant $y$-coordinates left and right of this threshold. The MSEs on both sides of the threshold are weighted by the number of instances on their side. Thus, the cost function is given by
$$J(k,t_k)=\frac{m_{\rm left}}{m}{\rm MSE}_{\rm left}+\frac{m_{\rm right}}{m}{\rm MSE}_{\rm right}\,,$$
where ${\rm MSE}_{\rm node}=\sum_{i\in\rm{node}}(y_{\rm node}-y^{(i)})^2$ and $y_{\rm node}=\frac{1}{m_{\rm node}}\sum_{i\in{\rm node}}y^{(i)}$.<br>
Scikit-Learn's *DecisionTreeRegressor* class works in a similar way: in each node, the *value* attribute gives the mean of the current node. Then the node splits the test set at some threshold. How this threshold is found is not discussed in the book. Possibly, the goal is to minimize the MSE in the child nodes in a similar way as CART does.

In [4]:
# see Github link
import numpy as np
np.random.seed(42)
m = 200
X = np.random.rand(m, 1)           # create 200 instance of x- ...
y = 4 * (X - 0.5) ** 2
y = y + np.random.randn(m, 1) / 10 # ... and y-coordinates (the latter are noisy)
# train a DecisionTreeRegressor
from sklearn.tree import DecisionTreeRegressor
tree_reg = DecisionTreeRegressor(max_depth=2)
print(tree_reg.fit(X, y))
# export using graphviz (similar to above procedure but with different filename)
export_graphviz(
        tree_reg,
        out_file=image_path("regression_tree.dot"),
        feature_names=["x1"],
        rounded=True,
        filled=True
    )

DecisionTreeRegressor(criterion='mse', max_depth=2, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=None, splitter='best')


## Instability
page 177<br>
Decision Trees usually employ orthogonal decision boundaries. This is because the number of features is often equal or at least closely related to the dimension of the data. However, classes are often identified by combinations of features such that the decision boundaries might not be orthogonal.<br>
Assume two classes can be perfectly separated with exactly one decision boundary. Then, a simple rotation of the data by 45° makes it hard for Decision Trees to find the line that separates the two classes because this diagonal line has to be constructed by many perpendicular lines and corners that approximate that diagonal. Prinicipal Component Analysis (PCA, see chapter 8) is often succesfull in removing such instabilities.<br>
Moreover, removing only a single instance can lead to a very different tree. So there is some *instability* with respect to added or removed data. Random Forests (chapter 7) can reduce this instability by training many trees and averaging over their predictions.
## Exercises
page 178
### 1.-6.
Solutions are shown in Appendix A of the book and in the separate notebook *ExercisesWithoutCode*.
### 7.
Train and fine-tune a DecisionTree for the moons dataset.
- Generate a moons dataset using *make_moons(n_samples=10000, noise=0.4)*.
- Split it into a training set and a test set using *train_test_split()*.
- Use grid search with cross-validation (with the help of the *GridSearchCV* class) to find good hyperparemeter values for a *DecisionTreeClassifier*. Hint: try various values for *max_leaf_nodes*.
- Train it on the full training set using these hyperparameters, and measure your model's performance on the test set. You should get roughly 85% to 87% accuracy.

Start with generating some moons data, splitting it into training and testing sets, and using grid search.

In [5]:
# generate moons data (see chapter 5)
from sklearn.datasets import make_moons
X, y = make_moons(n_samples=10000, noise=0.4, random_state=42)
# split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# import grid search
from sklearn.model_selection import GridSearchCV
params = {'max_leaf_nodes': list(range(2, 100)), # vary the number of nodes and the minimum ...
          'min_samples_split': [2, 3, 4]}        # ... number of samples necessary in a node for splitting
# train a DecisionTreeClassifier using grid search
grid_search_cv = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1, verbose=1)
grid_search_cv.fit(X_train, y_train)

Fitting 3 folds for each of 294 candidates, totalling 882 fits


/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  55 tasks      | elapsed:    6.4s
[Parallel(n_jobs=-1)]: Done 882 out of 882 | elapsed:   18.5s finished


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=DecisionTreeClassifier(class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort=False, random_state=42,
                                              splitter='best'),
             iid='warn', n_jobs=-1,
             param_grid={'max_leaf_nodes': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                            13, 14, 15,

Now use the best model and check its accuracy.

In [6]:
# take the best model
print(grid_search_cv.best_estimator_)
# import accuracy_score
from sklearn.metrics import accuracy_score
# make predictions
y_pred = grid_search_cv.predict(X_test)
# check the accuracy
print(accuracy_score(y_test, y_pred))

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=17,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=42, splitter='best')
0.8695


Indeed, the accuracy is above 85% and almost 87%.
### 8.
Grow a forest.
- Continuing the previous exercise, generate 1000 subsets of the training set, each containing 100 instances selected randomly. Hint: you can use Scikit-Learn's *ShuffleSplit* class for this.
- Train one Decision Tree on each subset, using the best hyperparameter values found above. Evaluate these 1000 Decision Trees on the test set. Since they were trained on smaller sets, these Decision Trees will likely perform worse than the first Decision Tree, achieving only about 80% accuracy.
- Now comes the magic. For each test set instance, generate the predicions of the 1000 Decision Trees, and keep only the most fequent prediction (you can use SciPy's *mode()* function for this). This gives you *majority-vote predictions * over the test set.
- Evaluate these predictions on the test set: you should obtain a slightly higher accuracy than your first model (about 0.5 to 1.5% higher). Congratulations, you have trained a Random Forest classifier!

Start with the first two tasks!

In [7]:
from sklearn.model_selection import ShuffleSplit
n_trees = 1000    # total number of mini sets
n_instances = 100 # number of instances per mini set
mini_sets = []    # container for the mini sets
rs = ShuffleSplit(n_splits=n_trees, test_size=len(X_train) - n_instances, random_state=42) # keep the training ...
for mini_train_index, mini_test_index in rs.split(X_train): # ... instances out of the test set; loop through ...
    X_mini_train = X_train[mini_train_index]                # ... the 1000 lists with indices for the training ...
    y_mini_train = y_train[mini_train_index]                # ... sets (features and labels) ...
    mini_sets.append((X_mini_train, y_mini_train))          # ... and put them in the container for the mini sets
# establish a forest by taking a thousand (untrained) copies of the best algorithm found in the previous exercise
from sklearn.base import clone
forest = [clone(grid_search_cv.best_estimator_) for _ in range(n_trees)] # this is the initial forest
accuracy_scores = []                                                     # container for accuracy scores
# training each tree in the forest and storing their accuracy scores
for tree, (X_mini_train, y_mini_train) in zip(forest, mini_sets):
    tree.fit(X_mini_train, y_mini_train)    
    y_pred = tree.predict(X_test)
    accuracy_scores.append(accuracy_score(y_test, y_pred))
np.mean(accuracy_scores)                                                 # average accuracy score

0.8054499999999999

The average accuracy is only slightly above 80%. That's not that great. Now, take care of the remaining two tasks!

In [8]:
# establish a container for storing all the predictions of all the classifiers ...
Y_pred = np.empty([n_trees, len(X_test)], dtype=np.uint8)
# ... and put all the predictions of all the classifiers inside
for tree_index, tree in enumerate(forest):
    Y_pred[tree_index] = tree.predict(X_test)
    # use SciPy's mode() function to classify according to the majority vote from the 1000 classifiers ...
from scipy.stats import mode
y_pred_majority_votes, n_votes = mode(Y_pred, axis=0)
# ... and check the accuracy of this "ensemble" classification
accuracy_score(y_test, y_pred_majority_votes.reshape([-1]))

0.872

The result is significantly better: more than 87% are correct!